In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import soccerdata as sd

[01/22/26 22:40:03] INFO     No custom team name replacements found. You can configure these in       ]8;id=243321;file://C:\Users\jakub\anaconda3\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=594125;file://C:\Users\jakub\anaconda3\Lib\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\jakub\soccerdata\config\teamname_replacements.json.                          

                    INFO     Custom league dict loaded from                                          ]8;id=294561;file://C:\Users\jakub\anaconda3\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=89163;file://C:\Users\jakub\anaconda3\Lib\site-packages\soccerdata\_config.py#195\195]8;;\
                             C:\Users\jakub\soccerdata\config\league_dict.json.                                    

In [ ]:
#za pomocą biblioteki soccerdata bedziemy pobierac dane ze strony fbref
fbref=sd.FBref(leagues="ENG-Premier League", seasons=['1920','2021','2122', '2223','2324', '2425' ])

In [ ]:
#wybieramy tabele ze staystykami, ktore nas interesuja
df_shooting=fbref.read_team_match_stats(stat_type='shooting')
df_schedule=fbref.read_team_match_stats(stat_type='schedule')
df_gs_creation=fbref.read_team_match_stats(stat_type='goal_shot_creation')

In [ ]:
df_shooting.columns

In [ ]:
df_schedule.columns

In [ ]:
df_gs_creation.columns

In [ ]:
df_schedule.to_csv('df_schedule.csv')
df_shooting.to_csv('df_shooting.csv')
df_gs_creation.to_csv('df_gs_creation.csv')

In [ ]:

#resetujemy index, żeby 'team', 'league', 'season', 'game' napewno stały się kolumnami

sched = df_schedule.reset_index()
shoot = df_shooting.reset_index()
creat = df_gs_creation.reset_index()

#wybieramy i zmieniamy nazwy kolumn aby były łatwiejsze do odczytu
base_df = sched[['date', 'team', 'opponent', 'game', 'GF', 'GA', 'match_report']].copy()
base_df.columns = ['date', 'team', 'opponent', 'game', 'goals_scored', 'goals_conceded', 'match_report']

#z tabeli shooting wybieramy statystyki, ktore nas interesuja, przez multi indexy uzywamy krotek
cols_shoot = shoot.loc[:, [
    ('match_report', ''), 
    ('team', ''),
    ('Expected', 'npxG'), 
    ('Standard', 'SoT')
]]
#spłaszczamy nazwy
cols_shoot.columns = ['match_report', 'team', 'npxg_created', 'sot_for']

#z tabeli gs_creation wybieramy statystyki, ktore nas interesuja, przez multi indexy uzywamy krotek
cols_creat = creat.loc[:, [
    ('match_report', ''), 
    ('team', ''),
    ('SCA Types', 'SCA')
]]
cols_creat.columns = ['match_report', 'team', 'sca_for']

#łączymy w całośc, używamy match_raport jako uniklanego klucza
df_full = pd.merge(base_df, cols_shoot, on=['match_report', 'team'], how='left')
df_full = pd.merge(df_full, cols_creat, on=['match_report', 'team'], how='left')

#formatowanie daty
df_full['date'] = pd.to_datetime(df_full['date'])
df_full = df_full.sort_values(['team', 'date'])

print("Wymiary po złączeniu:", df_full.shape)
print(df_full.head())

In [ ]:
# Tworzymy kopię, żeby przygotować dane z perspektywy PRZECIWNIKA
opponent_stats = df_full[['date', 'match_report', 'team', 'npxg_created', 'sot_for', 'sca_for']].copy()

# Zmieniamy nazwy kolumn na "conceded" (stracone/przeciwko)
opponent_stats.columns = ['date', 'match_report', 'opponent', 'npxg_conceded', 'sot_against', 'sca_against']

# Łączymy główną tabelę z tabelą przeciwnika
# Klucz: data, match_report oraz (ważne!) team w głównej = opponent w tabeli statystyk
df_final = pd.merge(
    df_full, 
    opponent_stats, 
    left_on=['date', 'match_report', 'opponent'], 
    right_on=['date', 'match_report', 'opponent'],
    how='left'
)

# Czyścimy ewentualne duplikaty kolumn, jeśli jakieś powstały
# (w tym przypadku nie powinny, bo precyzyjnie dobraliśmy kolumny)

print("Gotowe kolumny do modelu:")
print(df_final.columns.tolist())

In [ ]:
# Lista metryk, dla których liczymy średnią z 5 ostatnich meczów
metrics = [
    'goals_scored', 'goals_conceded', 
    'npxg_created', 'npxg_conceded', 
    'sca_for', 'sca_against',
    'sot_for', 'sot_against'
]

for metric in metrics:
    df_final[f'avg_{metric}_5'] = df_final.groupby('team')[metric].transform(
        lambda x: x.shift(1).rolling(window=5, min_periods=3).mean()
    )

# Dni od ostatniego meczu
df_final['days_since_last'] = df_final.groupby('team')['date'].diff().dt.days.fillna(30)

# Usuwamy pierwsze mecze, gdzie nie ma średnich
df_train = df_final.dropna(subset=[f'avg_goals_scored_5'])

In [ ]:
df_train

In [ ]:

import statsmodels.api as sm
import json

# --- 1. PRZYGOTOWANIE DANYCH ---

# Reset indeksów, żeby mieć pewność, że 'team', 'match_report' itd. są kolumnami
sched = df_schedule.reset_index()
shoot = df_shooting.reset_index()
creat = df_gs_creation.reset_index()

# A. Baza z terminarza (df_schedule)
# Potrzebujemy: kto, z kim, kiedy, wynik i ID meczu (match_report)
base_df = sched[['date', 'team', 'opponent', 'venue', 'GF', 'GA', 'match_report']].copy()
base_df.columns = ['date', 'team', 'opponent', 'venue', 'goals_scored', 'goals_conceded', 'match_report']

# B. Strzały (df_shooting) - bierzemy npxG (jakość sytuacji) i SoT (celne strzały)
# Uwaga: Odwołujemy się do krotek (tuples) bo to MultiIndex
cols_shoot = shoot.loc[:, [
    ('match_report', ''), 
    ('team', ''),
    ('Expected', 'npxG'), 
    ('Standard', 'SoT')
]]
cols_shoot.columns = ['match_report', 'team', 'npxg', 'sot']

# C. Kreatywność (df_gs_creation) - bierzemy SCA (akcje tworzące strzały)
cols_creat = creat.loc[:, [
    ('match_report', ''), 
    ('team', ''),
    ('SCA Types', 'SCA')
]]
cols_creat.columns = ['match_report', 'team', 'sca']

# --- 2. ŁĄCZENIE (MERGE) ---

# Łączymy bazę ze strzałami
df_full = pd.merge(base_df, cols_shoot, on=['match_report', 'team'], how='left')
# Łączymy z kreatywnością
df_full = pd.merge(df_full, cols_creat, on=['match_report', 'team'], how='left')

# Formatowanie daty i sortowanie
df_full['date'] = pd.to_datetime(df_full['date'])
df_full = df_full.sort_values(['team', 'date'])

# Upewnijmy się, że liczby są liczbami (czasem są stringami)
cols_to_numeric = ['goals_scored', 'goals_conceded', 'npxg', 'sot', 'sca']
for col in cols_to_numeric:
    df_full[col] = pd.to_numeric(df_full[col], errors='coerce').fillna(0)

print("Połączone dane (pierwsze 5 wierszy):")
print(df_full.head())

In [ ]:
#DODANIE STATYSTYK RYWALA (CONCEDED)

# kopia tabeli, która posłuży jako "dane przeciwnika"
opponent_stats = df_full[['date', 'match_report', 'team', 'npxg', 'sot', 'sca']].copy()

# zmieniamy nazwy na "conceded" (stracone)
opponent_stats.columns = ['date', 'match_report', 'opponent', 'npxg_conceded', 'sot_conceded', 'sca_conceded']

# łączymy: Szukamy wiersza, gdzie data i mecz są te same, ale 'team' w tabeli obok to nasz 'opponent'
df_model_data = pd.merge(
    df_full, 
    opponent_stats, 
    left_on=['date', 'match_report', 'opponent'], 
    right_on=['date', 'match_report', 'opponent'],
    how='inner' # inner, żeby mieć tylko pełne pary meczowe
)

# dodajemy flagę czy mecz był u siebie (1 = Home, 0 = Away)
df_model_data['is_home'] = df_model_data['venue'].apply(lambda x: 1 if x == 'Home' else 0)

print("\nDane z kolumnami defensywnymi:")
print(df_model_data[['team', 'opponent', 'npxg', 'npxg_conceded']].head())

In [ ]:
#OBLICZANIE ŚREDNICH (ROLLING AVERAGES) 

metrics = ['goals_scored', 'goals_conceded', 'npxg', 'npxg_conceded', 'sot', 'sot_conceded', 'sca', 'sca_conceded']

# dla każdej metryki tworzymy kolumnę "avg_METRYKA_5"
for metric in metrics:
    df_model_data[f'avg_{metric}_5'] = df_model_data.groupby('team')[metric].transform(
        lambda x: x.shift(1).rolling(window=5, min_periods=3).mean()
    )

# obliczamy dni odpoczynku (opcjonalnie, czasem pomaga)
df_model_data['days_since_last'] = df_model_data.groupby('team')['date'].diff().dt.days.fillna(7)

# PRZYGOTOWANIE DO TRENINGU 

# usuwamy pierwsze mecze, gdzie średnie są NaN (bo nie było 5 meczów wstecz)
train_df = df_model_data.dropna().copy()

print(f"\nLiczba meczów gotowych do treningu: {len(train_df)}")

In [ ]:
# TRENOWANIE MODELU 

# rozdzielamy na zbiór Home i Away
home_matches = train_df[train_df['is_home'] == 1]
away_matches = train_df[train_df['is_home'] == 0]

# MODEL DLA GOLI GOSPODARZA 
# Y = Gole strzelone przez gospodarza
# X = Atak Gospodarza (bramki, npxg, sca) + Obrona Gościa (npxg_conceded, goals_conceded)
X_home = home_matches[[
    'avg_goals_scored_5', 
    'avg_npxg_5', 
    'avg_sca_5', 
    'avg_goals_conceded_5', # To tak naprawdę obrona GOSPODARZA 
    'npxg_conceded' # OBRONA PRZECIWNIKA 
]]


opponent_avgs = train_df[['date', 'match_report', 'opponent', 'avg_npxg_5', 'avg_goals_conceded_5']].rename(columns={
    'opponent': 'team_join_key', # Żeby połączyć po nazwie rywala
    'avg_npxg_5': 'opp_avg_npxg_5',
    'avg_goals_conceded_5': 'opp_avg_goals_conceded_5'
})

# Dla każdego meczu Home, musimy pobrać statystyki Away.
final_train = pd.merge(
    home_matches,
    train_df[['date', 'match_report', 'team', 'avg_goals_conceded_5', 'avg_npxg_conceded_5']],
    left_on=['date', 'match_report', 'opponent'],
    right_on=['date', 'match_report', 'team'],
    suffixes=('', '_opp')
)

# MODEL HOME
features_home = ['avg_goals_scored_5', 'avg_npxg_5', 'avg_sca_5', 
                 'avg_goals_conceded_5_opp', 'avg_npxg_conceded_5_opp']

y_home = final_train['goals_scored']
X_home = final_train[features_home]
X_home = sm.add_constant(X_home)

model_home = sm.GLM(y_home, X_home, family=sm.families.Poisson()).fit()

# MODEL AWAY
final_train_away = pd.merge(
    away_matches,
    train_df[['date', 'match_report', 'team', 'avg_goals_conceded_5', 'avg_npxg_conceded_5']],
    left_on=['date', 'match_report', 'opponent'],
    right_on=['date', 'match_report', 'team'],
    suffixes=('', '_opp')
)

# Cechy są te same 
y_away = final_train_away['goals_scored']
X_away = final_train_away[features_home] # Używamy tych samych nazw kolumn dla wygody
X_away = sm.add_constant(X_away)

model_away = sm.GLM(y_away, X_away, family=sm.families.Poisson()).fit()

print("Modele wytrenowane!")
print(model_home.summary())

In [ ]:

from scipy.stats import poisson

# FUNKCJA POMOCNICZA DO OBLICZANIA KURSÓW
def calculate_odds(lambda_home, lambda_away):
    max_goals = 10
    prob_home = 0.0
    prob_draw = 0.0
    prob_away = 0.0

    for h in range(max_goals + 1):
        for a in range(max_goals + 1):
            p = poisson.pmf(h, lambda_home) * poisson.pmf(a, lambda_away)
            if h > a:
                prob_home += p
            elif h == a:
                prob_draw += p
            else:
                prob_away += p

    # Marża bukmachera (np. 5%)
    margin = 0.95 
    
    # Zabezpieczenie przed dzieleniem przez 0
    odd_1 = (1 / prob_home) * margin if prob_home > 0 else 1.0
    odd_x = (1 / prob_draw) * margin if prob_draw > 0 else 1.0
    odd_2 = (1 / prob_away) * margin if prob_away > 0 else 1.0
    
    return odd_1, odd_x, odd_2

# GŁÓWNA FUNKCJA PREDYKCJI
def predict_match_python(home_team, away_team, df_data, model_h, model_a):
    
    # Pobieramy ostatni wiersz z danymi dla obu drużyn
    try:
        # Bierzemy ostatni dostępny mecz (iloc[-1]) dla danej drużyny
        stats_home = df_data[df_data['team'] == home_team].iloc[-1]
        stats_away = df_data[df_data['team'] == away_team].iloc[-1]
    except IndexError:
        print(f"Błąd: Nie znaleziono danych dla {home_team} lub {away_team}")
        return

    
    input_home = pd.DataFrame({
        'const': [1.0],
        'avg_goals_scored_5': [stats_home['avg_goals_scored_5']],
        'avg_npxg_5': [stats_home['avg_npxg_5']],
        'avg_sca_5': [stats_home['avg_sca_5']],
        'avg_goals_conceded_5_opp': [stats_away['avg_goals_conceded_5']], # Statystyka GOŚCIA wchodzi jako OPP
        'avg_npxg_conceded_5_opp': [stats_away['avg_npxg_conceded_5']]   # Statystyka GOŚCIA wchodzi jako OPP
    })

    # Przygotowanie danych wejściowych dla modelu GOŚCIA
    
    input_away = pd.DataFrame({
        'const': [1.0],
        'avg_goals_scored_5': [stats_away['avg_goals_scored_5']],
        'avg_npxg_5': [stats_away['avg_npxg_5']],
        'avg_sca_5': [stats_away['avg_sca_5']],
        'avg_goals_conceded_5_opp': [stats_home['avg_goals_conceded_5']], # Statystyka GOSPODARZA wchodzi jako OPP
        'avg_npxg_conceded_5_opp': [stats_home['avg_npxg_conceded_5']]   # Statystyka GOSPODARZA wchodzi jako OPP
    })

    # Predykcja (Lambda)
    lambda_h = model_h.predict(input_home)[0]
    lambda_a = model_a.predict(input_away)[0]

    # Kursy
    o1, ox, o2 = calculate_odds(lambda_h, lambda_a)

    # Wyświetlanie wyników
    print(f"\n PROGNOZA: {home_team} vs {away_team}")
    print(f"-------------------------------------------")
    print(f"Przewidywane gole: {home_team} ({lambda_h:.2f}) - {away_team} ({lambda_a:.2f})")
    print(f"-------------------------------------------")
    print(f"KURS 1: {o1:.2f}")
    print(f"KURS X: {ox:.2f}")
    print(f"KURS 2: {o2:.2f}")
    print(f"-------------------------------------------")
    
    # Dodatkowa analiza 
    print("Dlaczego taki wynik?")
    print(f"Forma ataku {home_team}: {stats_home['avg_npxg_5']:.2f} npxG/mecz")
    print(f"Forma obrony {away_team}: {stats_away['avg_npxg_conceded_5']:.2f} npxG stracone/mecz")
    print(f"Forma ataku {away_team}: {stats_away['avg_npxg_5']:.2f} npxG/mecz")
    print(f"Forma obrony {home_team}: {stats_home['avg_npxg_conceded_5']:.2f} npxG stracone/mecz")



# Przykładowe pary do sprawdzenia:
predict_match_python('Chelsea', 'Southampton', train_df, model_home, model_away)
predict_match_python('Arsenal', 'Southampton', train_df, model_home, model_away)
predict_match_python('Southampton', 'Crystal Palace', train_df, model_home, model_away) # Mecz słabszych drużyn

In [ ]:
df_final['team'].value_counts()

In [ ]:
import json

# Tworzymy słownik z wagami
final_model_data = {
    "home_coeffs": model_home.params.to_dict(),
    "away_coeffs": model_away.params.to_dict()
}

# Zapisujemy do pliku
with open('model_fbref.json', 'w') as f:
    json.dump(final_model_data, f, indent=4)

print("Plik 'model_fbref.json' gotowy! Zawiera wagi dla npxG i SCA.")

In [ ]:
df_train.columns

In [ ]:

def generate_java_specs(df_input):
    # Tworzymy kopię, żeby nie psuć oryginału
    df = df_input.copy()
    
   
    numeric_cols = ['goals_scored', 'npxg_created', 'sca_for', 'sot_for', 'npxg_conceded']
    
    for col in numeric_cols:
        # Zamień na liczby, błędy zamień na NaN, a puste na 0
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    # OCENA SIŁY DRUŻYN
    # Grupujemy po drużynach, żeby ocenić ich siłę (średnie npxG)
    team_strength = df.groupby('team')['npxg_created'].mean().sort_values(ascending=False)
    
    # Dzielimy na 3 koszyki
    n = len(team_strength)
    top_teams = team_strength.index[:int(n*0.25)]       # Top 25%
    weak_teams = team_strength.index[int(n*0.75):]      # Dół 25%
    mid_teams = team_strength.index[int(n*0.25):int(n*0.75)] # Reszta
    
    #  FUNKCJA LICZĄCA WIDEŁKI
    def get_ranges(teams, tier_name):
        subset = df[df['team'].isin(teams)]
        
        return {
            "TIER": tier_name,
            # Gole (input do generatora statystyk po meczu)
            "Goals (zakres)": f"{subset['goals_scored'].quantile(0.1):.0f} - {subset['goals_scored'].quantile(0.95):.0f}",
            
            # Statystyki (input do modelu przed meczem - średnie)
            "npxG (created)": f"{subset['npxg_created'].quantile(0.1):.2f} - {subset['npxg_created'].quantile(0.9):.2f}",
            "SCA (created)":  f"{subset['sca_for'].quantile(0.1):.1f} - {subset['sca_for'].quantile(0.9):.1f}",
            "SoT (created)":  f"{subset['sot_for'].quantile(0.1):.1f} - {subset['sot_for'].quantile(0.9):.1f}",
            
            # Obrona
            "npxG (conceded)": f"{subset['npxg_conceded'].quantile(0.1):.2f} - {subset['npxg_conceded'].quantile(0.9):.2f}"
        }

    # GENEROWANIE RAPORTU 
    stats = []
    stats.append(get_ranges(top_teams, "TOP "))
    stats.append(get_ranges(mid_teams, "MID "))
    stats.append(get_ranges(weak_teams, "WEAK "))
    
    results = pd.DataFrame(stats)
    
    print("--- SPECYFIKACJA DLA GENERATORA W JAVIE ---")
    print("Kolego, użyj tych zakresów w StatsGeneratorze:\n")
    print(results.to_markdown(index=False))
    
    # Korelacja
    avg_sca_per_xg = (df['sca_for'] / df['npxg_created']).replace([np.inf, -np.inf], np.nan).mean()
    print(f"\n WSKAZÓWKA DLA JAVY: Średnio na 1.0 npxG przypada ok. {avg_sca_per_xg:.1f} akcji (SCA).")


generate_java_specs(df_train)